# Setep 5: Machine Learning Model

<br>
Clean up any values left from any previous steps

In [79]:
#Quick clean up
def cleanup():
    for name in dir():
        if not name.startswith('_'):
            del globals()[name]
cleanup()

In [40]:
#Load information from prevous steps
import pandas as pd

fields = ['Type', 'IE', 'NS', 'FT', 'PJ', 'Posts_Lemmatized']
mbti_Dataset = pd.read_pickle('mbti_Dataset2.pkl')
mbti_Dataset = mbti_Dataset.filter(fields)
mbti_Dataset.head()

,Type,IE,NS,FT,PJ,Posts_Lemmatized
0,INFJ,I,N,F,J,"[, moment, sportscenter, top, ten, play, prank, lifechanging, experience, life, repeat, today, may, perc, experience, immerse, last, thing, friend, posted, facebook, committing, suicide, next, day, rest, peace, hello, sorry, hear, distress, natural, relationship, perfection, time, every, moment, existence, try, figure, hard, time, time, growth, welcome, stuff, game, set, match, prozac, wellbrutin, least, thirty, minute, moving, leg, mean, moving, sitting, desk, chair, weed, moderation, maybe..."
1,ENTP,E,N,T,P,"[finding, lack, post, alarming, sex, boring, position, often, example, girlfriend, currently, environment, creatively, use, cowgirl, missionary, enough, giving, new, meaning, game, theory, hello, grin, take, converse, flirting, acknowledge, presence, return, word, smooth, wordplay, cheeky, grin, lack, balance, hand, eye, coordination, real, iq, test, score, internet, iq, test, funny, score, higher, like, former, response, thread, mention, believe, iq, test, banish, know, vanish, site, year, ..."
2,INTP,I,N,T,P,"[good, one, course, say, know, blessing, curse, absolutely, positive, best, friend, could, amazing, couple, count, yes, could, madly, love, case, reconciled, feeling, thank, link, socalled, tisi, loop, stem, current, topicobsession, deadly, like, stuck, thought, mind, wanders, circle, feel, truly, terrible, noticed, peculiar, vegetation, look, grass, dozen, different, plant, specie, imagine, hundred, year, later, whenif, soil, smith, â, never, one, ever, often, find, spotting, face, marble, ..."
3,INTJ,I,N,T,J,"[dear, enjoyed, conversation, day, esoteric, gabbing, nature, universe, idea, every, rule, social, code, arbitrary, construct, created, dear, sub, long, time, see, sincerely, alpha, none, type, hurt, deep, existential, way, want, part, probably, sliding, scale, depends, individual, preference, like, everything, humanity, draco, malfoy, also, would, say, either, either, though, stacking, somewhat, arbitrary, distinction, make, believe, core, indicates, primary, motivation, hand, every, action..."
4,ENTJ,E,N,T,J,"[fired, another, silly, misconception, approaching, logically, going, key, unlocking, whatever, think, entitled, nobody, want, approached, b, guy, really, want, go, superduperlongass, vacation, cmon, guy, bos, listen, get, even, approached, logically, everything, never, mind, go, permanent, vacation, two, month, would, crazy, idea, really, best, employee, may, cooking, want, reliable, asset, gone, long, employer, lol, like, view, unsolicited, victim, sometimes, really, like, impoverished, ra..."


## 5-1: Split our data into training and test

In [71]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(mbti_Dataset['Posts_Lemmatized'], mbti_Dataset[['IE' , 'NS', 'FT', 'PJ']], test_size=0.2)#20% of dataset is test set    


X_train.to_pickle('X_train')
X_train.to_pickle('X_test')
X_train.to_pickle('Y_train')
X_train.to_pickle('Y_test')
#np.save('Y_train', Y_train)#Saving results
#np.save('Y_test', Y_test)#Saving results
#np.save('X_train', Y_train)#Saving results
#np.save('X_test', Y_test)#Saving results

In [78]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy import sparse

def lemmatizing(text):
    """Lemmatizing the input text using WordNet and NLTK package"""
    NLTK_WNL = nltk.WordNetLemmatizer()
    text_Lem = [NLTK_WNL.lemmatize(word) for word in text]
    return(text_Lem)


def clean_and_load():
    for name in dir():
        if not name.startswith('_'):
            del globals()[name]
    global X_train, X_test, Y_train, Y_test
    X_train = pd.read_pickle('X_train')
    X_test = pd.read_pickle('X_test')
    Y_train = pd.read_pickle('Y_train')
    Y_test = pd.read_pickle('Y_test')


##TFIDF
#tfidf_vect = TfidfVectorizer(analyzer=lemmatizing)
#tfidf_vect_fit = tfidf_vect.fit(X_train)
#
#tfidf_train = tfidf_vect_fit.transform(X_train)
#tfidf_test = tfidf_vect_fit.transform(X_test)
#
#X_train_tfidf = pd.DataFrame(tfidf_train.toarray())
#X_test_tfidf = pd.DataFrame(tfidf_test.toarray())
#
#np.save('X_train_tfidf', X_train_tfidf)#Saving results
#np.save('X_test_tfidf', X_test_tfidf)#Saving results



#CountVectorizer
#count_vect = CountVectorizer(analyzer=lemmatizing)
#count_vect_fit = count_vect.fit(X_train)
#
#count_train = count_vect_fit.transform(X_train)
#count_test = count_vect_fit.transform(X_test)
#
#X_train_count = pd.DataFrame(count_train.toarray())
#X_test_count = pd.DataFrame(count_test.toarray())
#
#np.save('X_train_count', X_train_count)#Saving results
#np.save('X_test_count', X_test_count)#Saving results


#N-gram
ngram_vect = CountVectorizer(ngram_range=(1,3)) #Running N-Gram on full dataset (Max = 3)
X_train = X_train.apply(lambda x: ' '.join(map(str,x)))
ngram_vect_fit = ngram_vect.fit(X_train)

ngram_train = ngram_vect_fit.transform(X_train)
ngram_test = ngram_vect_fit.transform(X_test)

X_train_ngram = pd.DataFrame(ngram_train.toarray())
X_test_ngram = pd.DataFrame(ngram_test.toarray())

np.save('X_train_ngram', X_train_count)#Saving results
np.save('X_test_ngram', X_test_count)#Saving results


In [ ]:
import numpy as np
from scipy import sparse

#Saving results from previous step:
np.save('X_train_tfidf', X_test_tfidf)
np.save('X_test_tfidf', X_test_tfidf)

np.save('X_train_count', X_train_count)
np.save('X_test_count', X_test_count)

np.save('X_train_ngram', X_train_ngram)
np.save('X_test_ngram', X_test_ngram)

np.save('Y_train', Y_train)
np.save('Y_test', Y_test)

## 5-1: Random Forest Model
### 5-1-1: Explorering Random Forest with Holdout test set + grid-search
#### 5-1-1-1 : Split data into training and test set
#### 5-1-1-2 : Train vectorizes on training set and use that to transform test set.

In [17]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_rows', None, 'display.max_colwidth', 500)


def rfClassifier_GridSearch(X_test_set, X_train_set, Lable, n_est, depth, feature_type, is_print = True):
    if Lable == 'IE':
        s_Lable = 'I'
        predict_Lable = 'Introverts'
    elif Lable == 'NS':
        s_Lable = 'N'
        predict_Lable = 'Intuitives'
    elif Lable == 'FT':
        s_Lable = 'F'
        predict_Lable = 'Feelers'
    elif Lable == 'PJ':
        s_Lable = 'P'
        predict_Lable = 'Perceivers'

    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth ,n_jobs=-1)#Max depth of tree is 20
    rf_model = rf.fit(X_train_set, Y_train[Lable])
    Y_pred = rf_model.predict(X_test_set)

    precision, recall, fscore, support = score(Y_test[Lable], Y_pred, pos_label=s_Lable, average='binary')
    if is_print:
        print('Being ' + predict_Lable + ' using ' + feature_type + ':'  ,
        'Estimators: {} / Max_Depth: {} --> Precision: {} / Recall: {} / Accuracy: {}'.format(
                                                        n_est,
                                                        depth,
                                                        round(precision, 3),
                                                        round(recall, 3), 
                                                        round((Y_pred==Y_test[Lable]).sum() / len(Y_pred),3)))
    return([predict_Lable, feature_type, n_est, depth, precision, recall, round((Y_pred==Y_test[Lable]).sum() / len(Y_pred),3)])


classes = ['IE' , 'NS', 'FT', 'PJ']
rf_result = []
Y_test = sparse.load_npz('Y_test.npz')
Y_train = sparse.load_npz('Y_train.npz')

for feature_type in ['TFIDF', 'CountVectorizer', 'N-gram']:
    if feature_type == 'TFIDF':
        X_test_set = sparse.load_npz('X_test_tfidf.npz')
        X_train_set = sparse.load_npz('X_train_tfidf.npz')
    elif feature_type == 'CountVectorizer':
        X_test_set = sparse.load_npz('X_test_count.npz')
        X_train_set = sparse.load_npz('X_train_count.npz')
    elif feature_type == 'N-gram':
        X_test_set = sparse.load_npz('X_test_ngram.npz')
        X_train_set = sparse.load_npz('X_train_ngram.npz')
    for item in classes:
        for n_est in [10, 50, 100]:
            for depth in [10, 20, 30, None]:
                rf_result.append(rfClassifier_GridSearch(X_test_set = X_test_set, X_train_set = X_train_set, Lable = item, n_est=n_est, depth=depth, feature_type=feature_type,is_print = True))


pd.DataFrame(rf_result, columns= ['Type', 'Method', 'Estimators', 'Max_Depth', 'Precision', 'Recall', 'Accuracy']).to_csv('RF_Holdout_Result.csv')
pd.DataFrame(rf_result, columns= ['Type', 'Method', 'Estimators', 'Max_Depth', 'Precision', 'Recall', 'Accuracy'])

Being Introverts using TFIDF: Estimators: 10 / Max_Depth: 10 --> Precision: 0.768 / Recall: 1.0 / Accuracy: 0.768
Being Introverts using TFIDF: Estimators: 10 / Max_Depth: 20 --> Precision: 0.768 / Recall: 1.0 / Accuracy: 0.768
Being Introverts using TFIDF: Estimators: 10 / Max_Depth: 30 --> Precision: 0.769 / Recall: 0.996 / Accuracy: 0.768
Being Introverts using TFIDF: Estimators: 10 / Max_Depth: None --> Precision: 0.782 / Recall: 0.953 / Accuracy: 0.76
Being Introverts using TFIDF: Estimators: 50 / Max_Depth: 10 --> Precision: 0.768 / Recall: 1.0 / Accuracy: 0.768
Being Introverts using TFIDF: Estimators: 50 / Max_Depth: 20 --> Precision: 0.768 / Recall: 1.0 / Accuracy: 0.768
Being Introverts using TFIDF: Estimators: 50 / Max_Depth: 30 --> Precision: 0.768 / Recall: 1.0 / Accuracy: 0.768
Being Introverts using TFIDF: Estimators: 50 / Max_Depth: None --> Precision: 0.769 / Recall: 1.0 / Accuracy: 0.769
Being Introverts using TFIDF: Estimators: 100 / Max_Depth: 10 --> Precision: 0.76

,Type,Method,Estimators,Max_Depth,Precision,Recall,Accuracy
0,Introverts,TFIDF,10,10.0,0.767723,1.000000,0.768
1,Introverts,TFIDF,10,20.0,0.768166,1.000000,0.768
2,Introverts,TFIDF,10,30.0,0.769275,0.996246,0.768
3,Introverts,TFIDF,10,NaN,0.781538,0.953453,0.760
4,Introverts,TFIDF,50,10.0,0.767723,1.000000,0.768
5,Introverts,TFIDF,50,20.0,0.767723,1.000000,0.768
6,Introverts,TFIDF,50,30.0,0.767723,1.000000,0.768
7,Introverts,TFIDF,50,NaN,0.768609,1.000000,0.769
8,Introverts,TFIDF,100,10.0,0.767723,1.000000,0.768
9,Introverts,TFIDF,100,20.0,0.767723,1.000000,0.768


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd

pd.set_option('display.max_colwidth', 500)

def RF_Evaluation(param, X_Features, dataset_PD, lable, is_print = True):
    rf = RandomForestClassifier()
    gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)#cv=5 means 5 folde validation
    gs_fit = gs.fit(X_Features, dataset_PD)
    if is_print:
        print(pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[['param_max_depth',
        'param_n_estimators', 'std_test_score', 'mean_test_score', 'rank_test_score']][0:5])

    return(pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[['param_max_depth',
    'param_n_estimators', 'std_test_score', 'mean_test_score', 'rank_test_score']][0:5])


param = {'n_estimators' : [10, 150, 300],
        'max_depth' : [30, 60, 90, None]}

classes = ['IE' , 'NS', 'FT', 'PJ']
for item in classes:
    for key, X_Features in features_Dic.items():
        RF_Evaluation(param, X_Features, mbti_Dataset[item], item)
        print(key)


   param_max_depth param_n_estimators  std_test_score  mean_test_score  \
6               90                 10        0.004380         0.771873   
0               30                 10        0.001094         0.770605   
10            None                150        0.000565         0.770490   
11            None                300        0.000461         0.770375   
7               90                150        0.000461         0.770259   

    rank_test_score  
6                 1  
0                 2  
10                3  
11                4  
7                 5  
Count Vectorizer
   param_max_depth param_n_estimators  std_test_score  mean_test_score  \
10            None                150        0.000936         0.770259   
8               90                300        0.000431         0.770144   
11            None                300        0.000672         0.770144   
4               60                150        0.000365         0.770029   
5               60                30

## 5-2: Gradient Boosting Model
### 5-2-1: Gradient Boosting with Holdout test set

In [2]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
from datetime import datetime


pd.set_option('display.max_rows', None, 'display.max_colwidth', 500)


def GBoosting_GridSearch(X_Features, dataset_PD, test_Szie, Lable, feature_type , n_est, depth, lr ,is_print = True):
    if Lable == 'IE':
        Lable = 'I'
        predict_Lable = 'Introverts'
    elif Lable == 'NS':
        Lable = 'N'
        predict_Lable = 'Intuitives'
    elif Lable == 'FT':
        Lable = 'F'
        predict_Lable = 'Feelers'
    elif Lable == 'PJ':
        Lable = 'P'
        predict_Lable = 'Perceivers'
    X_train, X_test, Y_train, Y_test = train_test_split(X_Features, dataset_PD, test_size=test_Szie) # 20% of our dataset is test set
    gb = GradientBoostingClassifier(n_estimators=n_est, max_depth=depth , learning_rate=lr)#Max depth of tree is 20
    gb_model = gb.fit(X_train, Y_train)
    Y_pred = gb_model.predict(X_test)
    precision, recall, fscore, support = score(Y_test, Y_pred, pos_label=Lable, average='binary')
    if is_print:
        print('Being ' + predict_Lable + ' using ' + feature_type + ': '  ,'Estimators: {} / Max_Depth: {} / Learning_Rate: {} --> Precision: {} / Recall: {} / Accuracy: {}'.format(
                                                        n_est,
                                                        depth,
                                                        lr,
                                                        round(precision, 3),
                                                        round(recall, 3), 
                                                        round((Y_pred==Y_test).sum() / len(Y_pred),3)))
    return([predict_Lable, feature_type, n_est, depth, lr, precision, recall, round((Y_pred==Y_test).sum() / len(Y_pred),3)])



classes = ['IE' , 'NS', 'FT', 'PJ']
gb_result = []

for item in classes:
    for key, X_Features in features_Dic.items():
        for n_est in [50, 100, 150]:
            for depth in [3, 7, 11, 15]:
                for lr in [0.01, 0.10, 1.00]:
                    gb_result.append(GBoosting_GridSearch(X_Features, mbti_Dataset[item], 0.2, item, key, n_est, depth, lr, is_print=False))
                    print('Moving to {} as Learning_Rate at: '.format(lr), datetime.now())
                print('Moving to {} as Max_Depth at: '.format(depth), datetime.now())
            print('Moving to {} estimators at: '.format(n_est), datetime.now())
        print('Moving to {} feature at: '.format(key), datetime.now())
    print('Moving to {} class at: '.format(item), datetime.now())

pd.DataFrame(gb_result, columns= ['Type', 'Method', 'Estimators', 'Max_Depth', 'Learning_Rate','Precision', 'Recall', 'Accuracy']).to_csv('GB_Holdout_Result.csv')
pd.DataFrame(gb_result, columns= ['Type', 'Method', 'Estimators', 'Max_Depth', 'Learning_Rate','Precision', 'Recall', 'Accuracy']).head()


ving to 0.1 as Learning_Rate at:  2021-06-26 02:08:21.947917
Moving to 1.0 as Learning_Rate at:  2021-06-26 02:20:05.114699
Moving to 11 as Max_Depth at:  2021-06-26 02:20:05.114699
Moving to 0.01 as Learning_Rate at:  2021-06-26 02:48:08.762527
Moving to 0.1 as Learning_Rate at:  2021-06-26 03:05:06.819204
Moving to 1.0 as Learning_Rate at:  2021-06-26 03:20:12.301838
Moving to 15 as Max_Depth at:  2021-06-26 03:20:12.301838
Moving to 150 estimators at:  2021-06-26 03:20:12.301838
Moving to TFIDF Vectorizer feature at:  2021-06-26 03:20:12.301838
Moving to IE class at:  2021-06-26 03:20:12.301838
Moving to 0.01 as Learning_Rate at:  2021-06-26 03:21:31.333904
Moving to 0.1 as Learning_Rate at:  2021-06-26 03:22:50.244996
Moving to 1.0 as Learning_Rate at:  2021-06-26 03:24:07.262245
Moving to 3 as Max_Depth at:  2021-06-26 03:24:07.262245
Moving to 0.01 as Learning_Rate at:  2021-06-26 03:26:22.749736
Moving to 0.1 as Learning_Rate at:  2021-06-26 03:28:23.732490
Moving to 1.0 as Lear

,Type,Method,Estimators,Max_Depth,Learning_Rate,Precision,Recall,Accuracy
0,Introverts,Count Vectorizer,50,3,0.01,0.774640,1.000000,0.775
1,Introverts,Count Vectorizer,50,3,0.10,0.828499,0.973333,0.822
2,Introverts,Count Vectorizer,50,3,1.00,0.842327,0.891030,0.786
3,Introverts,Count Vectorizer,50,7,0.01,0.758083,0.999239,0.758
4,Introverts,Count Vectorizer,50,7,0.10,0.821705,0.960725,0.811


### 5-2-2: Evaluation Gradient Boosting Model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd

pd.set_option('display.max_colwidth', 500)


def GB_Evaluation(param, X_Features, dataset_PD, lable):
    gb = GradientBoostingClassifier()
    gs = GridSearchCV(gb, param, cv=5)#cv=5 meand 5 folde validation
    gs_fit = gs.fit(X_Features, dataset_PD)
    print(pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[['param_max_depth',
     'param_n_estimators', 'std_test_score', 'mean_test_score', 'rank_test_score']][0:5])

    return(pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending = False)[['param_max_depth',
     'param_n_estimators', 'std_test_score', 'mean_test_score', 'rank_test_score']][0:5])
  
param = {'n_estimators' : [50, 100, 150],
                'max_depth' : [7, 11, 15],
                'Learning_rate' : [0.1]}

classes = ['IE' , 'NS', 'FT', 'PJ']


for item in classes:
    for key, X_Features in features_Dic.items():
        GB_Evaluation(param, X_Features, mbti_Dataset[item], item)
        print(key)

In [ ]:
import winsound
frequency = 2500  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)